
# Staged Metaprogramming
I had a post somewehre on this point. That programming z3 in python is staged metaprogramming.
The meta system python can do partial evaluation.


Can I eval z3 expressions in python metalayer? the other options is quoting 
Getting more homicornicity




In [1]:
from z3 import *

In [ ]:
def mypow(x:int,n:int) -> int:
    assert n >= 0
    if n == 0:
        return 1
    else:
        x * mypow(x,n-1)

# accumulator version?

# string version
# strings are a universal but somewhat structure free rep of code.
Code = str
def mypow2(n:int, x:Code) -> Code:
    if x == 0:
        return "1"
    else:
        f"{x} * {mypow2(x,n-1)}"

mypow = lambda x,n: 1 if n <= 0 else x * mypow(x,n-1)

mypow = Function("mypow", IntSort(), IntSort(), IntSort())
mypow_def = ForAll([x,n], mypow(n, x) == If(n <= 0, 1, x * mypow(n-1, x)))

# Partially evaled
def mypow(x:ExprRef, n:int) -> ExprRef:
    if n == 0:
        return IntVal(1)
    else:
        return x * mypow(x,n-1)




# Def Eq
We are maintaining a definition thing.



Function("def_eq")?
Obviously too slippery (althoguh it is kind of meaningful if we so happen to hide the definition axioms)
but better is the quoted version. We can kind of reflect definitional equality as an axiom schema into quoted terms.
Function("def_eq", z3_term, z3_term)

Universes and metaprogramming layers feel similar ish. They're both a hiearchy of meta stuff.
Relation to justified smt?



In [ ]:
def defeq(t):
    if in kernel.defns[t.decl()]

# Quote


"oracle"

hilbert epsilon. "choice function". maybe that's more accurate? in the sense a formula is a set?



self mutating sat

continuations. self cegis

Related to the above, we can also build a replication of some of z3's ast inside itself. The `quote` function is a metalevel python notion.

https://arxiv.org/abs/1802.00405
qe hol-light

Quotation is rife with paradox, but I'm not sure that the way I've done it here can be? Since I haven't internalized quote into my logic.


What about having an internalized Formula and solver functions. Similar to formulog but more homoiconic. So it makes sense to have 3 layers or more.
Maybe a way of formulating 

smtlib macro
Homoiconic. Who's to say there isn't a higher level z3 in which we are formulas embedded into. Is there a way to request things of this higher z3?
python3 -m smtpp 
A magic # or something as a tick?


In [ ]:
BoolExpr = Datatype("BoolExpr")
BoolExpr.declare("BoolVal", ("val", BoolSort())) # splice kind of
BoolExpr.declare("Var", ("name", StringSort()))
BoolExpr.declare("And", ("lhs", BoolExpr), ("rhs", BoolExpr))
BoolExpr.declare("Or", ("lhs", BoolExpr), ("rhs", BoolExpr))
BoolExpr.declare("Not", ("arg", BoolExpr))
BoolExpr.declare("Quote", ("arg", BoolExpr))
BoolExpr = BoolExpr.create()

Model = ArraySort(StringSort(), BoolSort())
sat = Function("sat", BoolExpr, BoolSort()) # sat ~ exists model, interp(form, model)
eps = Function("eps", BoolExpr, Model) # eps ~ model, interp(form, model) = True
satisfies = Function("satisifies", BoolExpr, Model, BoolSort()) # M |= phi   relation.

# satisfies(f, eps(f)) = sat(f)
def cegis_loop(s : Solver):
    res = s.check()
    if = 
    m = s.model()


In [1]:
"""
unityped
Z3Expr = Datatype("Z3Expr")
Z3Expr.declare("IntVal", ("val", IntSort()))
Z3Expr.declare("Var", ("name", StringSort()))
Z3Expr.declare("And", ("lhs", Z3Expr), ("rhs", Z3Expr))
Z3Expr.declare("Or", ("lhs", Z3Expr), ("rhs", Z3Expr))
"""
BoolExpr = Datatype("BoolExpr")
BoolExpr.declare("BoolVal", ("val", BoolSort()))
BoolExpr.declare("Var", ("name", StringSort()))
BoolExpr.declare("And", ("lhs", BoolExpr), ("rhs", BoolExpr))
BoolExpr.declare("Or", ("lhs", BoolExpr), ("rhs", BoolExpr))
BoolExpr.declare("Not", ("arg", BoolExpr))
BoolExpr = BoolExpr.create()
# maybe reals is better
IntExpr = Datatype("IntExpr")
IntExpr.declare("IntVal", ("val", IntSort()))
IntExpr.declare("Var", ("name", StringSort()))
IntExpr.declare("Add", ("lhs", IntExpr), ("rhs", IntExpr))
IntExpr.declare("Sub", ("lhs", IntExpr), ("rhs", IntExpr))
IntExpr.declare("Quote", ("expr", IntExpr)
IntExpr = IntExpr.create()


Result = Datatype("Result")
Result.declare("sat")
Result.declare("unsat")
Result.declare("unknown")
Function("check-sat" BoolSort())

Value = Datatype("Value")
Value.declare("Int", ("int", IntSort()))
Value.declare("Bool", ("bool", BoolSort()))
Value = Value.create()
Model = ArraySort(Id, Value) # Or refactor to (ArraySort(id, Bool), ArraySort(Id, Real))
# Very similar to hilbert epsilon
Function("get-model", BoolExpr, Model)
booleval = Function("booleval", BoolExpr, Model)


def quote(e : ExprRef)-> ExprRef:
    match e.decl().name():
        case "IntVal": return Z3Expr.IntVal(e)
        #case "Var": return Z3Expr.Var(e)
        case "and": return Z3Expr.And(quote(e.arg(0)), quote(e.arg(1)))
        case "or": return Z3Expr.Or(quote(e.arg(0)), quote(e.arg(1)))
        case _: raise Exception(f"Unknown decl: {e.decl().name()}")

# eval = Function(Z3Expr, ArraySort(StringSort(), BoolSort()), BoolSort())
# eval_def = ForAll([e,env], eval(e) == If( ... ))

SyntaxError: '(' was never closed (3140188254.py, line 22)

# User defined propagators
https://microsoft.github.io/z3guide/programming/Example%20Programs/User%20Propagator/

If I have some things I've proven, can I reflect them into z3's mechanisms?

 plant config paper https://web.ist.utl.pt/nuno.lopes/pubs/plantconfig-cpaior.pdf

https://dl.acm.org/doi/abs/10.1007/978-3-031-24950-1_5 sat modulo custom thoeires

But putting regular solving in a loop and looking at models to see if secret constraints are violated is just as viable? It may not be performant.


Refinement typing via user propagator

https://link.springer.com/chapter/10.1007/978-3-031-43513-3_2 Non-Classical Logics in Satisfiability Modulo Theories https://github.com/CEisenhofer/ModalZ3


# Pattern matching on z3 asts


In [13]:
from typing import Optional, Iterable, Dict
def match_(t : AstRef, pat : AstRef, vars : Iterable[ExprRef] = []) -> Optional[Dict[ExprRef,ExprRef]]:
    subst = {}
    todo = [(t,pat)]
    while todo:
        t,pat = todo.pop()
        if pat in vars or is_var(pat): # allow var as pattern?
            if pat in subst:
                if subst[pat].eq(t):
                    pass
                else:
                    return None
            else:
                subst[pat] = t
        elif isinstance(t, QuantifierRef) or isinstance(pat, QuantifierRef):
            raise NotImplementedError
        else:
            thead, targs = t.decl(), t.children()
            phead, pargs = pat.decl(), pat.children()
            if thead != phead or len(targs) != len(pargs): # check sorts here? The decl might check that
                return None
            todo.extend(zip(targs, pargs))
    return subst

E = DeclareSort("Expr")
foo = Function("foo", E, E, E)
bar = Function("bar", E, E)
a,b,c = Consts("a b c", E)
x,y,z = Consts("x y z", E)
vars = {x,y,z}

assert match_(foo(x, bar(y)), foo(a, bar(b)), vars) == None
assert match_(foo(a, bar(b)), foo(x, bar(y)), vars) == {x:a, y:b}
assert match_(foo(a, bar(b)), foo(a, bar(b)), vars) == {}
assert match_(foo(a, bar(b)), foo(x,x), vars) == None
assert match_(foo(a, bar(a)), foo(x,bar(x)), vars) == {x:a}
match_(foo(a, bar(a)), Lambda([x],foo(x,bar(x))).body()) == {Var(0, E):a}


True

matching as an internalized construct.

Note in my 2019 blog post I used an interesting keyword arg based desing


In [ ]:
z3.DatatypeRef.match = lambda self, **kwargs: 

In [8]:
Var(0, IntSort())

AttributeError: 'ArithRef' object has no attribute 'body'

# Overloading


In [ ]:
# This is kind of oleg's thing
__add_methods = {}
#ExprRef.__add__ = lambda self,other: __add_methods[self.sort()](self, other)
#ExprRef.__mul__ = 

# could also use multi dissatch
def register_add(sort, func):
    add_methods[sort] = func
def deregister_add()

# Hmm. It disdpatch on the python type though. No the z3 sort. 
from functools import singledispatch
@singledispatch
def add(x,y):
    raise NotImplementedError("Unsupported type")
ExprRef.__add__ = add

# Is this too overwrought?

class ExprDispatch(): # Sort dispatch
    def __init__(self):
        self.methods = {}
    def register(self, sort, func):
        self.methods(sort) = func
    def __call__(self, *args, **kwargs):
        return self.methods[args[0].sort()](*args, **kwargs)

add = ExprDispatch()
ExprRef.__add__ = add

mul = ExprDispatch()
ExprRef.__mul__ = mul

sub = ExprDispatch()
ExprRef.__sub__ = sub

matmul = ExprDispatch()
ExprRef.__matmul__ = matmul


In [8]:
from IPython.core.display import display, HTML
display(HTML('<div width="100px"><hr></div>&forall;x'))

/tmp/ipykernel_1070664/3665299815.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [26]:
from IPython.core.display import HTML

html_content = """
<style>
  .proof-tree {
    font-family: 'DejaVu Sans', Arial, sans-serif;
    line-height: 1.5;
    color: white; /* Set text color to white for dark background */
  }
  .rule {
    display: flex;
    flex-direction: column;
    align-items: center;
    margin-bottom: 20px;
  }
  .premises {
    display: flex;
    justify-content: bottom;
    align-items: flex-end; 
  }
  .premises div {
    margin: 0 10px;
  }
  .conclusion {
    border-top: 1px solid white; /* Set border color to white for visibility */
    padding-top: 5px;
    text-align: bottom;
  }
</style>

<div class="proof-tree">

  <div class="rule">
    <div class="premises">
        <div class="rule">
        <div class="premises">
           <div class="conclusion">A</div>
           <div class="conclusion">B</div>
       </div>
       <div class="conclusion">A &#x2227; B</div>
    </div>
      <div class="conclusion">C</div>
    </div>
    <div class="conclusion">(A &#x2227; B) &#x2227; C</div>
  </div>
</div>
"""

HTML(html_content)


class Proof():
    def tree(self, depth=3):
        if depth <= 0:
            return "<div>...</div>"
        return f"""
        <div class="premises">
          {"\n".join(r.tree(depth-1) for r in self.reasons)}
        </div>
        <div class="conclusion">{repr(self.thm)}</div>
        """



# Database
Lemma search is super important. Also docs

Maybe I should make some kind of auto scraper.

Use AI to generate summaries

RAG



In [54]:
import knuckledragger.theories.Real as Real
from knuckledragger.kernel import is_proof
for n, p in Real.__dict__.items():
    if is_proof(p):
       display(n, HTML(p._repr_html_()))

dir(Real.plus_def.thm)


'plus_def'

'plus_0'

'plus_comm'

'plus_assoc'

'mul_def'

...

'mul_comm'

'mul_assoc'

'mul_distrib'

'abs_def'

['__add__',
 '__and__',
 '__bool__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '__xor__',
 '_repr_html_',
 'arg',
 'as_ast',
 'ast',
 'body',
 'children',
 'ctx',
 'ctx_ref',
 'decl',
 'eq',
 'from_string',
 'get_id',
 'hash',
 'is_exists',
 'is_forall',
 'is_lambda',
 'no_pattern',
 'num_args',
 'num_no_patterns',
 'num_patterns',
 'num_vars',
 'params',
 'pattern',
 'qid',
 'serialize',
 'sexpr',
 'skolem_id',
 'sort',
 'sort_kind',
 'translate',
 'use_pp',
 'var_name',
 'var_sort',
 'weight']

In [43]:
Real.plus_def.thm.use_pp()
repr(Real.plus_def.thm)
Real.plus_def._repr_html_()

'&#8870;&forall;x, y : plus(x, y) = x + y'

In [88]:
import sys
for modname,mod in sys.modules.items():
    if "knuckledragger" in modname:
        print(modname)
        for thmname, proof in mod.__dict__.items():
            if is_proof(proof):
                display(thmname, HTML(proof._repr_html_()))

__editable___knuckledragger_0_1_1_finder
knuckledragger.kernel
knuckledragger
knuckledragger.theories
knuckledragger.theories.Real


'plus_def'

'plus_0'

'plus_comm'

'plus_assoc'

...

'mul_1'

'mul_comm'

'mul_assoc'

'mul_distrib'

'abs_def'

# Simp
We can use match_ to do a simp routine, knuckledragger tactic.

Idea: use eggllg
https://egglog-python.readthedocs.io/latest/reference/bindings.html

https://fadoss.github.io/maude-bindings/

to prolog?

In [1]:
import maude
maude.init()
m = maude.getModule('NAT')
t = m.parseTerm('2 * 3')
t.reduce()
print(t)

6


In [6]:
sy = t.symbol()
str(sy)

's_'

In [ ]:
def maude_sort_to_z3_sort()

def maude_to_z3(t) -> ExprRef:
    f = t.symbol()
    Function(str(f), *[f.f.domainKind(i) for i in range(f.arity())], f.rangeSort())
    maude_to_z3(arg) for arg in t.arguments()

def z3_to_maude(t) -> maude.Term:
    # very similar to tptp

In [98]:
from z3 import *

Math = DeclareSort("Math")
add = Function("add", Math, Math, Math)
dir(add(x,y) == x)
(add(x,y) == x).decl().name()

'='

In [121]:
import egglog.bindings as eggbnd
from z3 import *

Math = DeclareSort("Math")
add = Function("add", Math, Math, Math)
zero = Const("zero", Math)
one =  Const("one", Math)

sig = [Math, add, zero, one]

x,y = Consts("x y", Math)
add(zero,one) == one
rules =[
ForAll([x], add(x,zero) == x),
ForAll([x,y], add(x,y) == add(y,x)),
]

def z3_to_egglog(e, vars=[]):
    if e in vars:
        return e.sexpr()
    else:
        head, args = e.decl(), e.children()
        return f"({head.name()} {' '.join(z3_to_egglog(a, vars) for a in args)})"

from knuckledragger.utils import open_binder
def simp(e, rules, sig, n=10):
    output = []
    #output.append("(declare-sort TERM)")
    for f in sig:
        if isinstance(f, z3.FuncDeclRef):
            dom = " ".join([f.domain(i).name() for i in range(f.arity())])
            output.append(f"(function {f.name()}  ({dom}) {f.range().name()})")
        elif isinstance(f, z3.SortRef):
            output.append(f"(datatype {f.name()})")
        elif isinstance(f, z3.ExprRef):
            output.append(f"(function {f.sexpr()} () {f.sort().name()})")
    for r in rules:
        assert r.is_forall()
        vs, r = open_binder(r)
        assert r.decl().name() == "="
        lhs, rhs = r.children()
        output.append(f"(rewrite {z3_to_egglog(lhs,vars=vs)} {z3_to_egglog(rhs, vars=vs)})")
    output.append(f"(let knuck_e {z3_to_egglog(e)})")
    output.append(f"(run {n})")
    output.append("(extract knuck_e)")
    return output


egraph = eggbnd.EGraph()
for c in simp(add(zero,one), rules, sig):
    print(c)
    commands = egraph.parse_program(c)
    res = egraph.run_program(*commands)
print(res)


(datatype Math)
(function add  (Math Math) Math)
(function zero () Math)
(function one () Math)
(rewrite (add X (zero )) X)
(rewrite (add X Y) (add Y X))
(let knuck_e (add (zero ) (one )))
(run 10)
(extract knuck_e)
['(one)']


# By Definitions

delta expansion.
Also a simp routine doing this would be nice.


Should I make a db of everything that comes through?
Maybe some config parameters?

I could also make the Kernel entities objects instead of global.



It's too bad that we lose the nice type definition of the manual style.
fact = define(  , lambda fact: If  , fix=True)

Scan for symbol being in rhs
aprove? NaTT?

Maybe lemma should not do the clever checking and that should be in a util wrapper.



In [ ]:

def decls(e : ExprRef): #get_sig. get_sig might also collect the types?
    if is_var(e):
        return
    else:
        yield e.decl()
        yield from c in e.children() for d in decls(c)
def by_defn(thm: z3.BoolRef, by=[]) -> Proof:  # delta expansion
    lemma(thm, by=[kernel.defn[decls(thm)]] + by)
    

# Lambda Eval

I've complained a bit before that it's crazy python doesn't have a good lambda library.
Well, it kind of does.


Lambda eval.
Z3 does do capture avoinding substitution. It can take care of de bruijn indices.
vs z3 simplify

Locally nameless matching. FreshConst + substitute makes this easy

Then we can use regular python pattern matching for lambda matching. Pretty cool!
I don't understand how the property mechanism works.

https://chargueraud.org/research/2009/ln/main.pdf 

In [51]:
"""
z3.ExprRef.head = property(lambda self: self.decl().kind())
z3.ExprRef.args = property(lambda self: [self.arg(i) for i in range(self.num_args())])
z3.ExprRef.__match_args__ = ["head", "args"]
z3.QuantifierRef.open_term = property(lambda self: vars = FreshConst() (return vars, subst(self.body, []))) 
z3.QuantifierRef.__match_args__ = ["open_term"]

z3.QuantifierRef.__matmul__ = lambda self, other: z3.substitute(self.body, zip([z3.Var(n) for n in range(len(other)) , other]))
"""
def open_binder(l : QuantifierRef):
    vars = []
    nvars = l.num_vars()
    for i in range(nvars):
        sort = l.var_sort(i)
        vars.append((Var(nvars - i - 1, sort) , FreshConst(sort, prefix=l.var_name(i))))
    return [x for (_,x) in vars], substitute(l.body(), vars)



open_binder(Lambda([x, y, z], foo(x,bar(bar(y)))))
open_binder(ForAll([x, y, z], foo(x,bar(bar(y))) == bar(z)))

def instan(f : QuantifierRef, *x):
    nvars = f.num_vars()
    assert len(x) == nvars
    vars = []
    for i in range(nvars):
        sort = f.var_sort(i)
        vars.append((Var(nvars - i - 1, sort), x[i]))
    return substitute(f.body(), *vars)

instan(ForAll([x, y, z], foo(x,bar(bar(y))) == bar(z)), a, b, bar(c))

# instn is substutie_vars?
# how much of this is also available from cvc5

foo(a, bar(bar(b))) == bar(bar(c))

In [53]:
help_simplify()

algebraic_number_evaluator (bool) simplify/evaluate expressions containing (algebraic) irrational numbers. (default: true)
arith_ineq_lhs (bool) rewrite inequalities so that right-hand-side is a constant. (default: false)
arith_lhs (bool) all monomials are moved to the left-hand-side, and the right-hand-side is just a constant. (default: false)
bit2bool (bool) try to convert bit-vector terms of size 1 into Boolean terms (default: true)
blast_distinct (bool) expand a distinct predicate into a quadratic number of disequalities (default: false)
blast_distinct_threshold (unsigned int) when blast_distinct is true, only distinct expressions with less than this number of arguments are blasted (default: 4294967295)
blast_eq_value (bool) blast (some) Bit-vector equalities into bits (default: false)
blast_select_store (bool) eagerly replace all (select (store ..) ..) term by an if-then-else term (default: false)
bv_extract_prop (bool) attempt to partially propagate extraction inwards (default: f

In [48]:
list(range(3,0,-1))

[3, 2, 1]

In [1]:
# It isn't all roses. Python's scoping rules are bonkers. It's intrinsically an imperative language.
match 42:
    case a:
        pass
print(a)

42


In [ ]:
def unify(a : AstRef, b : AstRef, vars):
    # or take in an existential equation.
    eqs = [(a,b)]
    subst = []
    sig = [] # miller unify?
    while eqs:
        a,b = eqs.pop()
        if a in vars:
            subst.append((a,b))
            map(lambda x: substitute(subst), eqs)
        elif b in vars:
            subst.append((b,a))
            map(lambda x: substitute(subst), eqs)
        


convert z3 ast to egglog for conditional simplification.

simp taking in z3 formula

https://microsoft.github.io/z3guide/programming/Example%20Programs/Formula%20Simplification/ examples of simplifying z3 expressions.


rules = []
def simp1(t, rules):
    for r in rules:
        match r:
            case ForAll(vars,l == r):
                subst = unify(l, r, vars)
                if subst != None:

def simp():





Term orderings


<Order.GT: 4>

ast vector and ast map are interesting.

class UnionFind():
    pass

The thing that kills me is too many design decisions. The great thing about just taking the z3 ast is it just is what it is. It is a pretty goodc design alkbeit imperfetc. In fact, it's so good it kind of inspired this whole line of thinking on my part.


https://www.philipzucker.com/programming-and-interactive-proving-with-z3py/

https://stackoverflow.com/questions/76270483/is-there-a-way-to-draw-bussproof-style-tree-diagram-in-jupyter-notebook



In [54]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Theorem():
    pass
__Theorem = Theorem
Theorem = None

@dataclass(frozen=True)
class Axiom(__Theorem):
    thm : BoolRef
#Axiom = None. # Actually. Go ahead

@dataclass(frozen=True)
class Lemma(__Theorem):
    thm : BoolRef
    by : list[Theorem]
    admit : bool
__Lemma = Lemma #This is silly? You can still get at it.
Lemma = None

def lemma(fm : BoolRef, by = [], admit = False):
    if admit:
        return __Lemma(fm, by, True)
    else:
        s = Solver()
        for n, h in enumerate(by):
            assert isinstance(h, __Theorem)
            s.add(h.thm)
            s.assert_and_track(h, f"by_{n}")
         
        s.assert_and_track(Not(fm), "goal")
        s.set("unsat_core", True)
        s.set("timeout", 1000)
        res = s.check()
        if res == unsat:
            # check the unsat core. Check it include goals
            core = s.unsat_core()
            return __Lemma(fm, by, False)
        elif res == sat:
            raise Exception("Lemma failed to prove", s.get_model())



In [ ]:
# a protocol style.
class Proof():
    thm : BoolRef
    reasons : list[Any]
    def check(self):
        pass

class Z3Lemma():
    def check(self):
        for i in self.reasons:
            i.check()
        lemma(self.thm, self.reasons)

In [ ]:
Theorem

# Converting to TPTP


In [53]:
def collect_sig(e : ExprRef):
    sig = set()
    todo = [e]
    while todo:
        e = todo.pop()
        if is_var(e):
            continue
        elif isinstance(e, QuantifierRef):
            todo.extend(e.body())
        else:    
          sig.add(e.decl())
          todo.extend(e.children())
    return sig

E = DeclareSort("E")
x,y,z = Ints("x y z")
z,w = Consts("z w", E)
collect_sig(And(z == w, w == z))

{==, And, w, z}

;;declarations
(declare-fun f (Int) Int)
;;axioms
;;goal
(assert (not (= x x)))
(check-set)


NameError: name 'even_pf' is not defined

In [57]:
def sig_tptp_decl(decl):
    name = decl.name()
    if name in ["and", "or", "=>", "=", "ite", "not"]:
        return None
    return "thf(typedecl, type, {} : {}).".format(name, z3_sort_tptp(decl.sort()))

for decl in collect_sig(And(z == w, w == z)):
    print(decl)
    print(sig_tptp_decl(decl))


Z3Exception: sort mismatch

In [54]:




x, y = Ints("x y")
z3_to_tptp(And(x == 1, y == 2))

'((x = 1) & (y = 2))'

In [85]:
e= Not(BoolVal(False))
e.sort_kind()
e.decl().kind() == z3.Z3_OP_NOT
f(x).decl().kind() == z3.Z3_OP_UNINTERPRETED
BoolVal(False).decl().kind() == z3.Z3_OP_FALSE

True

In [ ]:
_z3_op_to_tptp = {
    Z3_OP_TRUE: "$true",
    Z3_OP_FALSE: "$false",
    Z3_OP_EQ: "=",
    Z3_OP_DISTINCT: "Distinct",
    Z3_OP_ITE: "$ite",
    Z3_OP_AND: "&",
    Z3_OP_OR: "|",
    Z3_OP_IFF: "<=>",
    Z3_OP_XOR: "Xor",
    Z3_OP_NOT: "~",
    Z3_OP_IMPLIES: "=>",

    Z3_OP_IDIV: "/",
    Z3_OP_MOD: "%",
    Z3_OP_TO_REAL: "ToReal",
    Z3_OP_TO_INT: "ToInt",
    Z3_OP_POWER: "**",
    Z3_OP_IS_INT: "IsInt",
    Z3_OP_BADD: "+",
    Z3_OP_BSUB: "-",
    Z3_OP_BMUL: "*",
    Z3_OP_BOR: "|",
    Z3_OP_BAND: "&",
    Z3_OP_BNOT: "~",
    Z3_OP_BXOR: "^",
    Z3_OP_BNEG: "-",
    Z3_OP_BUDIV: "UDiv",
    Z3_OP_BSDIV: "/",
    Z3_OP_BSMOD: "%",
    Z3_OP_BSREM: "SRem",
    Z3_OP_BUREM: "URem",

    Z3_OP_EXT_ROTATE_LEFT: "RotateLeft",
    Z3_OP_EXT_ROTATE_RIGHT: "RotateRight",

    Z3_OP_SLEQ: "<=",
    Z3_OP_SLT: "<",
    Z3_OP_SGEQ: ">=",
    Z3_OP_SGT: ">",

    Z3_OP_ULEQ: "ULE",
    Z3_OP_ULT: "ULT",
    Z3_OP_UGEQ: "UGE",
    Z3_OP_UGT: "UGT",
    Z3_OP_SIGN_EXT: "SignExt",
    Z3_OP_ZERO_EXT: "ZeroExt",

    Z3_OP_REPEAT: "RepeatBitVec",
    Z3_OP_BASHR: ">>",
    Z3_OP_BSHL: "<<",
    Z3_OP_BLSHR: "LShR",

    Z3_OP_CONCAT: "Concat",
    Z3_OP_EXTRACT: "Extract",
    Z3_OP_BV2INT: "BV2Int",
    Z3_OP_ARRAY_MAP: "Map",
    Z3_OP_SELECT: "Select",
    Z3_OP_STORE: "Store",
    Z3_OP_CONST_ARRAY: "K",
    Z3_OP_ARRAY_EXT: "Ext",

    Z3_OP_PB_AT_MOST: "AtMost",
    Z3_OP_PB_AT_LEAST: "AtLeast",
    Z3_OP_PB_LE: "PbLe",
    Z3_OP_PB_GE: "PbGe",
    Z3_OP_PB_EQ: "PbEq",

    Z3_OP_SEQ_CONCAT: "Concat",
    Z3_OP_SEQ_PREFIX: "PrefixOf",
    Z3_OP_SEQ_SUFFIX: "SuffixOf",
    Z3_OP_SEQ_UNIT: "Unit",
    Z3_OP_SEQ_CONTAINS: "Contains",
    Z3_OP_SEQ_REPLACE: "Replace",
    Z3_OP_SEQ_AT: "At",
    Z3_OP_SEQ_NTH: "Nth",
    Z3_OP_SEQ_INDEX: "IndexOf",
    Z3_OP_SEQ_LAST_INDEX: "LastIndexOf",
    Z3_OP_SEQ_LENGTH: "Length",
    Z3_OP_STR_TO_INT: "StrToInt",
    Z3_OP_INT_TO_STR: "IntToStr",

    Z3_OP_SEQ_IN_RE: "InRe",
    Z3_OP_SEQ_TO_RE: "Re",
    Z3_OP_RE_PLUS: "Plus",
    Z3_OP_RE_STAR: "Star",
    Z3_OP_RE_OPTION: "Option",
    Z3_OP_RE_UNION: "Union",
    Z3_OP_RE_RANGE: "Range",
    Z3_OP_RE_INTERSECT: "Intersect",
    Z3_OP_RE_COMPLEMENT: "Complement",

    Z3_OP_FPA_IS_NAN: "fpIsNaN",
    Z3_OP_FPA_IS_INF: "fpIsInf",
    Z3_OP_FPA_IS_ZERO: "fpIsZero",
    Z3_OP_FPA_IS_NORMAL: "fpIsNormal",
    Z3_OP_FPA_IS_SUBNORMAL: "fpIsSubnormal",
    Z3_OP_FPA_IS_NEGATIVE: "fpIsNegative",
    Z3_OP_FPA_IS_POSITIVE: "fpIsPositive",
}

In [55]:
def z3_problem_to_tptp(probs : list[BoolRef]):
    tptp = []
    sig = {d for prob in probs for d in collect_sig(prob)}
    tptp.extend(map(sig_tptp_decl, sig))
    tptp.extend(map(z3_to_tptp, probs))
    return tptp

x, y, z = Ints("x y z")
z3_problem_to_tptp([x == y, y == z])

    


AttributeError: 'FuncDeclRef' object has no attribute 'sort'

In [112]:
Exists([x], x == x).is_exists()
substitute_vars(Lambda([x,y], x + y).body(), IntVal(3))
Select(Lambda([x], x + 1), 10)

AttributeError: 'QuantifierRef' object has no attribute 'domain_n'

In [69]:
x.decl().name()

'x'

In [73]:
# nice smt printing. Using solver kind of mangles it
from z3 import *
x = Int("x")
f = Function("f", IntSort(), IntSort())
x.decl().sexpr()

def sexpr_datatype(dt):
    print(f"(declare-datatypes ({dt.name()}))")
    for c in dt.constructor():
        print(f"({c.name()} {c.sort()})")
    #print(f"(declare-fun is_{dt.name()} (E) Bool)")
    #print(f"(assert (forall ((x E)) (= (is_{dt.name()} x) (or {c.name()} x))))")


import subprocess

def lemma_smt(thm, by=[], sig=[]):
    output = []
    output.append(";;declarations")
    for f in sig:
        output.append(f.sexpr())
    output.append(";;axioms")
    for e in by:
        output.append("(assert " + e.sexpr() + ")")
    output.append(";;goal")
    output.append("(assert " + Not(thm).sexpr() + ")")
    output.append("(check-set)")
    return output

def lemma_ext(thm, by=[], sig=[], command=None, timeout=1000):
    # TODO use better temporary file
    with open("/tmp/temp.smt2", "w") as f:
        smt = lemma_smt(thm, by=by, sig=sig)
        f.writelines(smt)
    command.append("/tmp/temp.smt2")
    res = subprocess.run(
        command,
        timeout=1.0,
        capture_output=True,
    )
    return res

def lemma_vampire(thm, by=[], sig=[], timeout=1000):
    return lemma_ext(thm, by=by, sig=sig, command=["vampire", "--output_mode", "smtcomp"], timeout=timeout)

lemma_vampire(f(7) == 7, sig=[f]).stdout

;;declarations
(declare-fun f (Int) Int)
;;axioms
;;goal
(assert (not (= x x)))
(check-set)


b'Running in auto input_syntax mode. Trying SMTLIB2\nsat\n'